<a href="https://colab.research.google.com/github/Matcap97/OBJECT_DETECTION/blob/main/notebooks/OBJECT_DETECTION_YOLOv11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### INSTALLAZIONE PACCHETTI

In [ ]:
!pip install torch torchvision ultralytics
!pip install import-ipynb

IMPORT PACCHETTI NECESSARI

In [22]:
from ultralytics import YOLO
import import_ipynb
import os
from google.colab import drive
import pandas as pd

import cv2
from PIL import Image

from ultralytics import YOLO

In [2]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
%cd "drive/MyDrive/OBJECT_DETECTION_YOLOv11"

/content/drive/MyDrive/OBJECT_DETECTION_YOLOv11


IMPORT MODELLO YOLOv11 NANO E SMALL PER TEST

In [8]:
model_n = YOLO("./models/yolo11n.pt")
model_s = YOLO("./models/yolo11s.pt")

In [9]:
im1 = Image.open("./immagini_test/bus.jpg")
new_size = (im1.width // 3, im1.height // 3)
img_resized = im1.resize(new_size)
img_resized

In [11]:
results = model_n.predict(source=im1, save=True)


0: 512x640 2 buss, 51.0ms
Speed: 26.7ms preprocess, 51.0ms inference, 419.0ms postprocess per image at shape (1, 3, 512, 640)
Results saved to runs/detect/predict


In [29]:
print('CLASSI SUPPORTATE')
pd.DataFrame.from_dict(model_n.names,orient='index',columns=['DESCRIZIONE'])

CLASSI SUPPORTATE


,DESCRIZIONE
0,person
1,bicycle
2,car
3,motorcycle
4,airplane
...,...
75,vase
76,scissors
77,teddy bear
78,hair drier


In [39]:
model_n.names.values()

dict_values(['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'])

In [42]:
model_n.model[:8]

TypeError: 'DetectionModel' object is not subscriptable